In [ ]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
% cd /content/

In [ ]:
!git clone https://github.com/tensorflow/tensor2tensor.git
#! pip install -q -U tensor2tensor==1.15.7

In [ ]:
%cd tensor2tensor
!pip install .

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [ ]:
#import tensorflow as tf
tf.__version__

In [ ]:
#import tensorflow as tf

# Enable Eager execution - useful for seeing the generated data.
tf.enable_eager_execution()

from tensor2tensor.utils import trainer_lib

# Set a seed so that we have deterministic outputs.
RANDOM_SEED = 301
trainer_lib.set_random_seed(RANDOM_SEED)

In [ ]:
#upload train data ligands file
from google.colab import files
files.upload()

In [ ]:
#upload train data proteins file
from google.colab import files
files.upload()

In [ ]:
!mv d2_tr_ligands.space_sep_seq ligands_train.space_sep_seq
!mv d2_tr_proteins.space_sep_seq proteins_train.space_sep_seq

In [ ]:
!ls

In [ ]:
import os

# Setup and create directories.
DATA_DIR = os.path.expanduser("/tmp/t2t/data")
OUTPUT_DIR = os.path.expanduser("/tmp/t2t/output")
TMP_DIR = os.path.expanduser("/tmp/t2t/tmp")
USR_DIR = os.path.expanduser("/tmp/t2t/usr")

# Create them.
tf.gfile.MakeDirs(DATA_DIR)
tf.gfile.MakeDirs(OUTPUT_DIR)
tf.gfile.MakeDirs(TMP_DIR)
tf.gfile.MakeDirs(USR_DIR)

In [ ]:
! mkdir transformer_scratch

In [ ]:
! cp /content/drive/MyDrive/IITPKD\ System\ Data/home/iit/Research\ Work/Experiments/Baselines/molecule_structure_generation/outputs/Dataset2/model/checkpoint /tmp/t2t/output
! cp /content/drive/MyDrive/IITPKD\ System\ Data/home/iit/Research\ Work/Experiments/Baselines/molecule_structure_generation/outputs/Dataset2/model/decode /tmp/t2t/output
! cp /content/drive/MyDrive/IITPKD\ System\ Data/home/iit/Research\ Work/Experiments/Baselines/molecule_structure_generation/outputs/Dataset2/model/e* /tmp/t2t/output
! cp /content/drive/MyDrive/IITPKD\ System\ Data/home/iit/Research\ Work/Experiments/Baselines/molecule_structure_generation/outputs/Dataset2/model/f* /tmp/t2t/output
! cp /content/drive/MyDrive/IITPKD\ System\ Data/home/iit/Research\ Work/Experiments/Baselines/molecule_structure_generation/outputs/Dataset2/model/g* /tmp/t2t/output
! cp /content/drive/MyDrive/IITPKD\ System\ Data/home/iit/Research\ Work/Experiments/Baselines/molecule_structure_generation/outputs/Dataset2/model/h* /tmp/t2t/output
! cp /content/drive/MyDrive/IITPKD\ System\ Data/home/iit/Research\ Work/Experiments/Baselines/molecule_structure_generation/outputs/Dataset2/model/model.ckpt-210937.* /tmp/t2t/output

In [ ]:
% cd /tmp/t2t/data

In [ ]:
#upload vocab file
from google.colab import files
files.upload()

In [ ]:
% cd /content

In [ ]:
%%writefile transformer_scratch/prob.py

import re

  
  
# `Problem` is the base class for any dataset that we want to add to T2T -- it
# unifies the specification of the problem for generating training data,
# training, evaluation and inference.
#
# All its methods (except `generate_data`) have reasonable default
# implementations.
#
# A sub-class must implement `generate_data(data_dir, tmp_dir)` -- this method
# is called by t2t-trainer or t2t-datagen to actually generate TFRecord dataset
# files on disk.
from tensor2tensor.data_generators import problem

# Certain categories of problems are very common, like where either the input or
# output is text, for such problems we define an (abstract) sub-class of
# `Problem` called `Text2TextProblem` -- this implements `generate_data` in
# terms of another function `generate_samples`. Sub-classes must override
# `generate_samples` and `is_generate_per_split`.
from tensor2tensor.data_generators import text_problems

# Every non-abstract problem sub-class (as well as models and hyperparameter
# sets) must be registered with T2T so that T2T knows about it and can look it
# up when you specify your problem on the commandline to t2t-trainer or
# t2t-datagen.
#
# One uses:
# `register_problem` for a new Problem sub-class.
# `register_model` for a new T2TModel sub-class.
# `register_hparams` for a new hyperparameter set. All hyperparameter sets
# typically extend `common_hparams.basic_params1` (directly or indirectly).
from tensor2tensor.utils import registry


# By default, when you register a problem (or model or hyperparameter set) the
# name with which it gets registered is the 'snake case' version -- so here
# the Problem class `ProteinSpecificLigandGeneration` will be registered with
# the name `protein_specific_ligand_generation`.
#
# One can override this default by actually assigning a name as follows:
# `@registry.register_problem("my_awesome_problem")`
#
# The registered name is specified to the t2t-trainer or t2t-datagen using the
# commandline flag `--problem`.
@registry.register_problem('protein_specific_ligand_generation')

# We inherit from `Text2TextProblem` which takes care of a lot of details
# regarding reading and writing the data to disk, what vocabulary type one
# should use, its size etc -- so that we need not worry about them, one can,
# of course, override those.


class ProteinSpecificLigandGeneration(text_problems.Text2TextProblem):
  

  # START: Methods we should override.

  # The methods that need to be overriden from `Text2TextProblem` are:
  # `is_generate_per_split` and
  # `generate_samples`.

  @property
  def is_generate_per_split(self):
    # If we have pre-existing data splits for (train, eval, test) then we set
    # this to True, which will have generate_samples be called for each of the
    # dataset_splits.
    #
    # If we do not have pre-existing data splits, we set this to False, which
    # will have generate_samples be called just once and the Problem will
    # automatically partition the data into dataset_splits.
    return False

  def generate_samples(self, data_dir, tmp_dir, dataset_split):
    # Here we are generating the data in-situ using the `sample_sentence`
    # function, otherwise we would have downloaded the data and put it in
    # `tmp_dir` -- and read it from that location.
    del tmp_dir

    # Unused here, is used in `Text2TextProblem.generate_data`.
    del data_dir

    # This would have been useful if `self.is_generate_per_split()` was True.
    # In that case we would have checked if we were generating a training,
    # evaluation or test sample. This is of type `problem.DatasetSplit`.
    del dataset_split

    
  #  drug_file = open ("drugs.smi.txt", "r")
  
  
   
    with open("/content/proteins_train.space_sep_seq") as file1, open("/content/ligands_train.space_sep_seq") as file2:
      for x, y in zip(file1, file2):
        inputs = x.strip()
        targets = y.strip()
        yield {"inputs":inputs,"targets":targets}
     
    
    
  # END: Methods we should override.

  # START: Overridable methods.

  @property
  def vocab_type(self):
    # We can use different types of vocabularies, `VocabType.CHARACTER`,
    # `VocabType.SUBWORD` and `VocabType.TOKEN`.
    #
    # SUBWORD and CHARACTER are fully invertible -- but SUBWORD provides a good
    # tradeoff between CHARACTER and TOKEN.
    return text_problems.VocabType.TOKEN
  



  @property
  def dataset_splits(self):
    # Since we are responsible for generating the dataset splits, we override
    # `Text2TextProblem.dataset_splits` to specify that we intend to keep
    # 80% data for training and 10% for evaluation and testing each.
    return [{
        "split": problem.DatasetSplit.TRAIN,
        "shards": 144000,
    }, {
        "split": problem.DatasetSplit.EVAL,
        "shards": 1,
    }, {
        "split": problem.DatasetSplit.TEST,
        "shards": 1,
    }]

 # END: Overridable methods.


In [ ]:
%%writefile transformer_scratch/__init__.py

from transformer_scratch import prob

# Very Important!
## Do these steps to successfully run this code on TF2 version:

0.   Change train:val:test splits in above code cell.
1.   Go to content/tensor2tensor/tensor2tensor/bin/t2t-trainer
2.   Update file with below code
    ```
    from __future__ import absolute_import
    from __future__ import division
    from __future__ import print_function

    from tensor2tensor.bin import t2t_trainer

    import tensorflow.compat.v1 as tf

    def main(argv):
        t2t_trainer.main(argv)

    if __name__ == "__main__":
        tf.disable_v2_behavior()
        tf.logging.set_verbosity(tf.logging.INFO)
        tf.app.run(main)
    ```
3.  Go to content/tensor2tensor/tensor2tensor/bin/t2t-decoder
4.   Update file with below code
    ```
    """t2t-decoder."""
    from __future__ import absolute_import
    from __future__ import division
    from __future__ import print_function

    #import tensorflow.compat.v1 as tf
    from tensor2tensor.bin import t2t_decoder
    import logging
    #import tensorflow as tf
    import tensorflow.compat.v1 as tf

    def main(argv):
        t2t_decoder.main(argv)

    if __name__ == "__main__":
        tf.disable_v2_behavior()
        tf.logging.set_verbosity(tf.logging.INFO)
        tf.app.run()
    ```
5.   Go to usr/local/bin/t2t-trainer
6.   Update file with below code
    ```
    from __future__ import absolute_import
    from __future__ import division
    from __future__ import print_function

    from tensor2tensor.bin import t2t_trainer

    import tensorflow.compat.v1 as tf

    def main(argv):
        t2t_trainer.main(argv)

    if __name__ == "__main__":
        tf.disable_v2_behavior()
        tf.logging.set_verbosity(tf.logging.INFO)
        tf.app.run(main)
    ```
7.  Go to usr/local/bin/t2t-decoder
8.   Update file with below code
    ```
    """t2t-decoder."""
    from __future__ import absolute_import
    from __future__ import division
    from __future__ import print_function

    #import tensorflow.compat.v1 as tf
    from tensor2tensor.bin import t2t_decoder
    import logging
    #import tensorflow as tf
    import tensorflow.compat.v1 as tf

    def main(argv):
        t2t_decoder.main(argv)

    if __name__ == "__main__":
        tf.disable_v2_behavior()
        tf.logging.set_verbosity(tf.logging.INFO)
        tf.app.run()
    ```



In [ ]:
%%writefile /content/tensor2tensor/tensor2tensor/bin/t2t-trainer
#!/usr/bin/env python
"""Trainer for Tensor2Tensor.

This script is used to train your models in Tensor2Tensor.

For example, to train a shake-shake model on MNIST run this:

t2t-trainer \
  --generate_data \
  --problem=image_mnist \
  --data_dir=~/t2t_data \
  --tmp_dir=~/t2t_data/tmp
  --model=shake_shake \
  --hparams_set=shake_shake_quick \
  --output_dir=~/t2t_train/mnist1 \
  --train_steps=1000 \
  --eval_steps=100
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from tensor2tensor.bin import t2t_trainer

import tensorflow.compat.v1 as tf

def main(argv):
   t2t_trainer.main(argv)

if __name__ == "__main__":
   tf.disable_v2_behavior()
   tf.logging.set_verbosity(tf.logging.INFO)
   tf.app.run(main)

In [ ]:
%%writefile /usr/local/bin/t2t-trainer
#!/usr/bin/env python
"""Trainer for Tensor2Tensor.

This script is used to train your models in Tensor2Tensor.

For example, to train a shake-shake model on MNIST run this:

t2t-trainer \
  --generate_data \
  --problem=image_mnist \
  --data_dir=~/t2t_data \
  --tmp_dir=~/t2t_data/tmp
  --model=shake_shake \
  --hparams_set=shake_shake_quick \
  --output_dir=~/t2t_train/mnist1 \
  --train_steps=1000 \
  --eval_steps=100
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from tensor2tensor.bin import t2t_trainer

import tensorflow.compat.v1 as tf

def main(argv):
   t2t_trainer.main(argv)

if __name__ == "__main__":
   tf.disable_v2_behavior()
   tf.logging.set_verbosity(tf.logging.INFO)
   tf.app.run(main)

In [ ]:
%%writefile /content/tensor2tensor/tensor2tensor/bin/t2t-decoder
#!/usr/bin/env python
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

#import tensorflow.compat.v1 as tf
from tensor2tensor.bin import t2t_decoder
import logging
#import tensorflow as tf
import tensorflow.compat.v1 as tf

def main(argv):
   t2t_decoder.main(argv)

if __name__ == "__main__":
   tf.disable_v2_behavior()
   tf.logging.set_verbosity(tf.logging.INFO)
   tf.app.run()

In [ ]:
%%writefile /usr/local/bin/t2t-decoder
#!/usr/bin/env python
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

#import tensorflow.compat.v1 as tf
from tensor2tensor.bin import t2t_decoder
import logging
#import tensorflow as tf
import tensorflow.compat.v1 as tf

def main(argv):
   t2t_decoder.main(argv)

if __name__ == "__main__":
   tf.disable_v2_behavior()
   tf.logging.set_verbosity(tf.logging.INFO)
   tf.app.run()

In [ ]:
import time
startTime = int(round(time.time() / 60))

In [ ]:
! t2t-datagen \
  --problem=protein_specific_ligand_generation \
  --data_dir=/tmp/t2t/data \
  --tmp_dir=/tmp/t2t/tmp \
  --t2t_usr_dir=/content/transformer_scratch

In [ ]:
! t2t-trainer \
  --model=transformer \
  --hparams_set=transformer_tiny \
  --hparams="batch_size=4096,max_length=0,label_smoothing=0,learning_rate_decay_scheme='noam',num_hidden_layers=4" \
  --problem=protein_specific_ligand_generation \
  --worker_gpu=1 \
  --train_steps=210937 \
  --data_dir=/tmp/t2t/data \
  --output_dir=/tmp/t2t/output \
  --t2t_usr_dir=/content/transformer_scratch

In [ ]:
endTime = int(round(time.time() / 60))
print(f'Total time for training: {endTime - startTime} min')

In [ ]:
%cd /tmp/t2t/output

In [ ]:
!ls

In [ ]:
!cp checkpoint /content/drive/MyDrive/IITPKD\ System\ Data/home/iit/Research\ Work/Experiments/Baselines/molecule_structure_generation/outputs/
!cp decode /content/drive/MyDrive/IITPKD\ System\ Data/home/iit/Research\ Work/Experiments/Baselines/molecule_structure_generation/outputs/
!cp -r e* /content/drive/MyDrive/IITPKD\ System\ Data/home/iit/Research\ Work/Experiments/Baselines/molecule_structure_generation/outputs/
!cp f* /content/drive/MyDrive/IITPKD\ System\ Data/home/iit/Research\ Work/Experiments/Baselines/molecule_structure_generation/outputs/
!cp g* /content/drive/MyDrive/IITPKD\ System\ Data/home/iit/Research\ Work/Experiments/Baselines/molecule_structure_generation/outputs/
!cp h* /content/drive/MyDrive/IITPKD\ System\ Data/home/iit/Research\ Work/Experiments/Baselines/molecule_structure_generation/outputs/
!cp model.ckpt-210937.* /content/drive/MyDrive/IITPKD\ System\ Data/home/iit/Research\ Work/Experiments/Baselines/molecule_structure_generation/outputs/

In [ ]:
!ls

In [ ]:
%cd /content/

In [ ]:
#upload test data proteins file
from google.colab import files
files.upload()

In [ ]:
!ls

In [ ]:
!cp d2_te_unique_proteins.txt /tmp/t2t/data

In [ ]:
%cd /tmp/t2t/data

In [ ]:
!ls

In [ ]:
import random

proteins = []
with open('d2_te_unique_proteins_run1.txt') as f:
    for line in f:
        if line.strip() not in proteins:
            proteins.append(line.strip())

with open('proteins_test1.space_sep_seq_unique', 'w+') as f:
    for p in proteins:
        sp = p
        print(sp)
        f.write(sp+'\n')

In [ ]:
% cd /content

In [ ]:
from tensor2tensor.bin import t2t_decoder

In [ ]:
import time
startTime = int(round(time.time() * 1000))

In [ ]:
! t2t-decoder \
  --hparams_set=transformer_tiny \
  --hparams="batch_size=4096,max_length=0,label_smoothing=0,learning_rate_decay_scheme='noam',num_hidden_layers=4" \
  --decode_hparams="beam_size=100,alpha=0.6,batch_size=4,write_beam_scores=False, return_beams=True" \
  --model=transformer \
  --problem=protein_specific_ligand_generation \
  --data_dir=/tmp/t2t/data \
  --output_dir=/tmp/t2t/output \
  --t2t_usr_dir=/content/transformer_scratch \
  --decode_from_file=/tmp/t2t/data/proteins_test1.space_sep_seq_unique \
  --decode_to_file=/content/drive/MyDrive/IITPKD\ System\ Data/home/iit/Research\ Work/Experiments/Baselines/molecule_structure_generation/outputs/Dataset2/generated_samples/protein.seq.decode.results_ds2_ep6000_beam100

In [ ]:
samples_dir = '/content/drive/MyDrive/IITPKD System Data/home/iit/Research Work/Experiments/Baselines/molecule_structure_generation/outputs/Dataset2/generated_samples/'
sample_size = 100

endTime = int(round(time.time()))
with open(samples_dir+f'samples_evaluation.txt', 'a+') as eval_f:
    eval_f.write(f'Total time for {sample_size} samples: {endTime - startTime} seconds\n')